# Melanoma Detection

This project is meant to be a test in image processing through a neural net and creates a classification algorithm using the topics covered in the Machine Learning Specialization on Coursera. It uses a dataset from kaggle that can be reached here:
https://www.kaggle.com/datasets/wanderdust/skin-lesion-analysis-toward-melanoma-detection

This model will use greyscale images of different types of skin blemishes to train on.

We first must import the required libraries

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot
from PIL import Image
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Data Collection

The next portion of this will consist of processing the images into a feature training set. This will also classify each based on the directory they come from. Since there are 3 classifiers, we will have 3 values of classification ranging from 0-2.

In [ ]:
image_dimension = (1022,767)
features = image_dimension[0]*image_dimension[1]
X_train = []
Y_train = []
data_dir = "./drive/MyDrive/ML/Melanoma/skin-lesions"
train_dir = f"{data_dir}/train"
val_dir = f"{data_dir}/valid"
test_dir = f"{data_dir}/test"

class_indicator = 0

for path in os.listdir(train_dir):
    print(f"*** Processing: {path} ***")
    for filename in os.listdir(f"{train_dir}/{path}"):
        print(f"Processing {path}-{filename}")
        im =  Image.open(f"{train_dir}/{path}/{filename}").convert("L")
        resized_im = im.resize(image_dimension)
        X_train.append(list(resized_im.getdata()))
        Y_train.append(class_indicator)
        im.close()
    class_indicator+=1


X_train = np.array(X_train)
Y_train = np.array(Y_train)
X_train = np.reshape(X_train, (-1, features))
Y_train = np.reshape(Y_train, (-1, 1))

print(f"X shape: {X_train.shape}")
print(f"Y shape: {Y_train.shape}")

In [3]:
p = np.random.permutation(X_train.shape[0])
X_train = X_train[p]
Y_train = Y_train[p]

In [ ]:
model = tf.keras.Sequential([
    tf.keras.Input(shape = (features,)),
    tf.keras.layers.Dense(units = 100, activation="relu"),
    tf.keras.layers.Dense(units = 100, activation="relu"),
    tf.keras.layers.Dense(units = 100, activation="relu"),
    tf.keras.layers.Dense(units = 100, activation="relu"),
    tf.keras.layers.Dense(units = 50, activation="relu"),
    tf.keras.layers.Dense(units = 10, activation="relu"),
    tf.keras.layers.Dense(units = 3, activation="linear")
])

model.summary()

model.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits="true"),
    optimizer = tf.keras.optimizers.Adam(0.01)
)

model.fit(X_train,Y_train, epochs=10)

predict = model.predict(X_train[0].reshape(-1,features))

print(f"Actual: {Y_train[0]}, Predicted: {predict}")